bước đầu tiên sử lý hết dữ liệu đầu vào 

In [23]:
#  chuẩn hóa giá trị đầu vào cho dataset 
import os
import torchaudio
import torch
import glob
from tqdm import tqdm

# --- CẤU HÌNH ---
# Thư mục chứa file wav gốc (Tiếng Việt)
folder = ['source' , 'target']
file = ['train' , 'test',  'valid']

TARGET_SAMPLE_RATE = 16000
# ----------------

def convert_to_16k(input_path, output_path):
    try:
        # Load audio
        waveform, sample_rate = torchaudio.load(input_path)

        # 1. Chuyển về Mono (nếu đang là Stereo)
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # 2. Resample về 16000Hz (nếu chưa đúng)
        if sample_rate != TARGET_SAMPLE_RATE:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=TARGET_SAMPLE_RATE)
            waveform = resampler(waveform)

        # Tạo thư mục cha nếu chưa có
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Lưu file
        torchaudio.save(output_path, waveform, TARGET_SAMPLE_RATE)
        
    except Exception as e:
        print(f"Lỗi file {input_path}: {e}")

def main():
    # Tìm tất cả file wav trong thư mục input (bao gồm cả thư mục con)
    for i in folder:
        for   j in file:
            INPUT_DIR = f"/home/khanh/Projects/KhoaLuan/my_data/{i}_audio/{j}/" 

            # Thư mục lưu j đã chuẩn hóa (Sẽ tự tạo nếu chưa có)
            OUTPUT_DIR = f"/home/khanh/Projects/KhoaLuan/my_data/{i}_wav_16k/{j}/"

            files = glob.glob(os.path.join(INPUT_DIR, "**/*.wav"), recursive=True)
        
            print(f"Tìm thấy {len(files)} ({i} in {j}) file audio. Bắt đầu xử lý...")

            for file_path in tqdm(files):
                # Tạo đường dẫn output tương ứng
                relative_path = os.path.relpath(file_path, INPUT_DIR)
                output_path = os.path.join(OUTPUT_DIR, relative_path)
                
                convert_to_16k(file_path, output_path)

            print("✅ Đã hoàn tất chuẩn hóa dữ liệu về 16kHz!")
            print(f"Dữ liệu mới nằm tại: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

Tìm thấy 2602 (source in train) file audio. Bắt đầu xử lý...


100%|██████████| 2602/2602 [00:23<00:00, 108.49it/s]


✅ Đã hoàn tất chuẩn hóa dữ liệu về 16kHz!
Dữ liệu mới nằm tại: /home/khanh/Projects/KhoaLuan/my_data/source_wav_16k/train/
Tìm thấy 647 (source in test) file audio. Bắt đầu xử lý...


100%|██████████| 647/647 [00:05<00:00, 128.10it/s]


✅ Đã hoàn tất chuẩn hóa dữ liệu về 16kHz!
Dữ liệu mới nằm tại: /home/khanh/Projects/KhoaLuan/my_data/source_wav_16k/test/
Tìm thấy 394 (source in valid) file audio. Bắt đầu xử lý...


...

✅ Đã hoàn tất chuẩn hóa dữ liệu về 16kHz!
Dữ liệu mới nằm tại: /home/khanh/Projects/KhoaLuan/my_data/target_wav_16k/train/
Tìm thấy 857 (target in test) file audio. Bắt đầu xử lý...


100%|██████████| 857/857 [00:14<00:00, 60.04it/s]


✅ Đã hoàn tất chuẩn hóa dữ liệu về 16kHz!
Dữ liệu mới nằm tại: /home/khanh/Projects/KhoaLuan/my_data/target_wav_16k/test/
Tìm thấy 361 (target in valid) file audio. Bắt đầu xử lý...


100%|██████████| 361/361 [00:06<00:00, 58.20it/s]

✅ Đã hoàn tất chuẩn hóa dữ liệu về 16kHz!
Dữ liệu mới nằm tại: /home/khanh/Projects/KhoaLuan/my_data/target_wav_16k/valid/


In [34]:
# Ví dụ cấu trúc thư mục của bạn: /data/en-vi/train/wav_en/ và /data/en-vi/train/wav_vi/

!python fairseq/examples/wav2vec/wav2vec_manifest.py my_data/source_wav_16k/train --dest manifest_temp/train_en --ext wav --valid-percent 0
!python fairseq/examples/wav2vec/wav2vec_manifest.py my_data/target_wav_16k/train --dest manifest_temp/train_vn --ext wav --valid-percent 0
# valid  
!python fairseq/examples/wav2vec/wav2vec_manifest.py my_data/target_wav_16k/valid --dest manifest_temp/dev_vn --ext wav --valid-percent 0
!python fairseq/examples/wav2vec/wav2vec_manifest.py my_data/source_wav_16k/valid  --dest manifest_temp/dev_en --ext wav --valid-percent 0

#  test 
!python fairseq/examples/wav2vec/wav2vec_manifest.py my_data/source_wav_16k/test --dest manifest_temp/test_en --ext wav --valid-percent 0
!python fairseq/examples/wav2vec/wav2vec_manifest.py my_data/target_wav_16k/test --dest manifest_temp/test_vn --ext wav --valid-percent 0



# !cp manifest_temp/dev_en/train.tsv manifest_temp/source/valid.tsv


In [35]:
!mkdir -p  manifest_temp/source
# source
!cp manifest_temp/dev_en/train.tsv manifest_temp/source/valid.tsv

!cp manifest_temp/train_en/train.tsv manifest_temp/source/train.tsv
!cp manifest_temp/test_en/train.tsv manifest_temp/source/test.tsv
# target
!mkdir -p manifest_temp/target
!cp manifest_temp/dev_vn/train.tsv manifest_temp/target/valid.tsv
!cp manifest_temp/train_vn/train.tsv manifest_temp/target/train.tsv
!cp manifest_temp/test_vn//train.tsv manifest_temp/target/test.tsv

In [36]:
# xoa thu muc cu di 
!rm -r manifest_temp/dev_vn/
!rm -r manifest_temp/dev_en/
!rm -r manifest_temp/train_en/
!rm -r manifest_temp/train_vn/
!rm -r manifest_temp/test_en/
!rm -r manifest_temp/test_vn/

In [37]:
# Tạo thư mục chứa feature tạm thời
!mkdir -p /home/khanh/Projects/KhoaLuan/hubert_feats

# Chạy lệnh dump (Lưu ý thay HUBERT_PATH đúng chỗ bạn vừa tải)
!python fairseq/examples/hubert/simple_kmeans/dump_hubert_feature.py \
    /home/khanh/Projects/KhoaLuan/manifest_temp/target \
    train \
    /home/khanh/Projects/KhoaLuan/checkpoints/mhubert_base_vp_en_es_fr_it3.pt \
    9 \
    1 \
    0 \
    /home/khanh/Projects/KhoaLuan/hubert_feats \
    --max_chunk 1600000

2026-01-22 10:36:25 | INFO | dump_hubert_feature | Namespace(tsv_dir='/home/khanh/Projects/KhoaLuan/manifest_temp/target', split='train', ckpt_path='/home/khanh/Projects/KhoaLuan/checkpoints/mhubert_base_vp_en_es_fr_it3.pt', layer=9, nshard=1, rank=0, feat_dir='/home/khanh/Projects/KhoaLuan/hubert_feats', max_chunk=1600000)
2026-01-22 10:36:26 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/khanh/Projects/KhoaLuan
2026-01-22 10:36:26 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/home/khanh/Projects/KhoaLuan/checkpoints/', 'fine_tuning': False, 'labels': ['km'], 'label_dir': '/checkpoint/wnhsu/experiments/hubert/kmeans/mhubert_vp_en_es_fr_it2_400k/en_es_fr.layer9.km500', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2

In [44]:
!python fairseq/examples/hubert/simple_kmeans/learn_kmeans.py \
    /home/khanh/Projects/KhoaLuan/hubert_feats \
    train \
    1 \
    /home/khanh/Projects/KhoaLuan/kmeans_model.joblib \
    100 \
    --percent 0.1

2026-01-22 00:21:40 | INFO | root | Namespace(feat_dir='/home/khanh/Projects/KhoaLuan/hubert_feats', split='train', nshard=1, km_path='/home/khanh/Projects/KhoaLuan/kmeans_model.joblib', n_clusters=100, seed=0, percent=0.1, init='k-means++', max_iter=100, batch_size=10000, tol=0.0, max_no_improvement=100, n_init=20, reassignment_ratio=0.0)
2026-01-22 00:21:41 | INFO | learn_kmeans | sampled 300 utterances, 157914 frames from shard 0/1
2026-01-22 00:21:42 | INFO | root | loaded feature with dimension (157914, 768)
Init 1/20 with method k-means++
Inertia for init 1/20: 2102450.5
Init 2/20 with method k-means++
Inertia for init 2/20: 2103638.75
Init 3/20 with method k-means++
Inertia for init 3/20: 2091183.75
Init 4/20 with method k-means++
Inertia for init 4/20: 2110545.5
Init 5/20 with method k-means++
Inertia for init 5/20: 2100426.25
Init 6/20 with method k-means++
Inertia for init 6/20: 2091635.25
Init 7/20 with method k-means++
Inertia for init 7/20: 2089754.25
Init 8/20 with method

In [48]:
!export PYTHONPATH=/home/khanh/Projects/KhoaLuan/fairseq:$PYTHONPATH

In [46]:
pwd

'/home/khanh/Projects/KhoaLuan'

In [ ]:
!PYTHONPATH= python  examples/textless_nlp/gslm/speech2unit/clustering/quantize_with_kmeans.py \
    --feature_type hubert \
    --kmeans_model_path /home/khanh/Projects/KhoaLuan/kmeans_model.joblib \
    --acoustic_model_path /home/khanh/Projects/KhoaLuan/hubert_base_ls960.pt \
    --layer 9 \
    --manifest_path /home/khanh/Projects/KhoaLuan/manifest_temp/train.tsv \
    --out_quantized_file_path /home/khanh/Projects/KhoaLuan/train.unit \
    --extension .wav

Traceback (most recent call last):
  File "/home/khanh/Projects/KhoaLuan/fairseq/examples/textless_nlp/gslm/speech2unit/clustering/quantize_with_kmeans.py", line 13, in <module>
    from examples.textless_nlp.gslm.speech2unit.clustering.utils import (
ModuleNotFoundError: No module named 'examples'


tạo cặp valid.unt và valid.tsv cho dữ liệu 

In [ ]:
#  tạo file valid.unit
export PYTHONPATH=$PYTHONPATH:$(pwd)/fairseq

python fairseq/examples/textless_nlp/gslm/speech2unit/clustering/quantize_with_kmeans.py \
    --feature_type hubert \
    --kmeans_model_path kmeans_model.joblib \
    --acoustic_model_path hubert_base_ls960.pt \
    --layer 9 \
    --manifest_path data-bin/valid.tsv \
    --out_quantized_file_path data-bin/valid.unit \
    --extension .wav

In [41]:
# 1. Đảm bảo file nguồn là file gốc (Lấy từ manifest_temp cho an toàn)
# Nếu bạn lỡ làm hỏng data-bin/valid.tsv rồi thì lệnh này sẽ khôi phục lại

# 2. Lấy thư mục gốc (Dòng đầu tiên của file)
!ROOT_DIR_VALID=$(head -n 1 data-bin/valid.tsv)

# 3. Lấy nội dung audio (Bỏ dòng đầu)
!tail -n +2 data-bin/valid.tsv > data-bin/valid_temp.tsv

# 4. Tạo Header chuẩn
!echo -e "id\taudio\tn_frames\ttgt_text\tspeaker" > data-bin/valid_st.tsv

# 5. GHÉP FILE (Đã sửa lỗi thiếu '/' và thêm -F'\t')
!paste data-bin/valid_temp.tsv data-bin/valid.unit | awk -F'\t' -v root="$ROOT_DIR_VALID" '{print "valid_"NR"\t"root"/"$1"\t"$2"\t"$3"\tspk1"}' >> data-bin/valid_st.tsv

# 6. Thay thế file cũ
!mv data-bin/valid_st.tsv data-bin/valid.tsv

# 7. Dọn dẹp
!rm data-bin/valid_temp.tsv

In [40]:
# 1. Đảm bảo file nguồn là file gốc (Lấy từ manifest_temp cho an toàn)
# Nếu bạn lỡ làm hỏng data-bin/valid.tsv rồi thì lệnh này sẽ khôi phục lại

# 2. Lấy thư mục gốc (Dòng đầu tiên của file)
!ROOT_DIR_VALID=$(head -n 1 data-bin/train.tsv)

# 3. Lấy nội dung audio (Bỏ dòng đầu)
!tail -n +2 data-bin/train.tsv > data-bin/train_temp.tsv

# 4. Tạo Header chuẩn
!echo -e "id\taudio\tn_frames\ttgt_text\tspeaker" > data-bin/train_st.tsv

# 5. GHÉP FILE (Đã sửa lỗi thiếu '/' và thêm -F'\t')
!paste data-bin/train_temp.tsv data-bin/train.unit | awk -F'\t' -v root="$ROOT_DIR_VALID" '{print "valid_"NR"\t"root"/"$1"\t"$2"\t"$3"\tspk1"}' >> data-bin/valid_st.tsv

# 6. Thay thế file cũ
!mv data-bin/train_st.tsv data-bin/train.tsv

# 7. Dọn dẹp
!rm data-bin/train_temp.tsv

In [42]:
#  chạy bằng code 
import os

# --- CẤU HÌNH ĐƯỜNG DẪN CỨNG (Sửa nếu đường dẫn máy bạn khác) ---
# Đây là thư mục chứa file audio gốc
TRAIN_AUDIO_ROOT = "/home/khanh/Projects/KhoaLuan/my_data/train"
VALID_AUDIO_ROOT = "/home/khanh/Projects/KhoaLuan/my_data/dev"

def fix_manifest_file(input_file, audio_root):
    print(f"🛠 Đang sửa file: {input_file}...")
    
    if not os.path.exists(input_file):
        print(f"❌ Không tìm thấy file: {input_file}")
        return

    with open(input_file, 'r') as f:
        lines = f.readlines()

    header = lines[0] # Giữ nguyên header
    new_lines = [header]
    
    for line in lines[1:]:
        parts = line.strip().split('\t')
        if len(parts) < 4: continue # Bỏ qua dòng lỗi

        # parts[1] hiện tại đang là: /filename.wav hoặc filename.wav
        filename = os.path.basename(parts[1]) 
        
        # Tạo đường dẫn tuyệt đối chuẩn xác
        abs_path = os.path.join(audio_root, filename)
        
        # Cập nhật lại cột audio
        parts[1] = abs_path
        
        # Ghép lại thành dòng hoàn chỉnh
        new_lines.append('\t'.join(parts) + '\n')

    # Ghi đè lại file cũ
    with open(input_file, 'w') as f:
        f.writelines(new_lines)
    
    print(f"✅ Đã sửa xong! Kiểm tra dòng đầu tiên:")
    print(new_lines[1].split('\t')[1]) # In thử đường dẫn audio dòng 1

if __name__ == '__main__':
    # Sửa tập Valid
    fix_manifest_file('data-bin/valid.tsv', VALID_AUDIO_ROOT)
    
    # Sửa luôn tập Train cho chắc ăn
    fix_manifest_file('data-bin/train.tsv', TRAIN_AUDIO_ROOT)

🛠 Đang sửa file: data-bin/valid.tsv...
✅ Đã sửa xong! Kiểm tra dòng đầu tiên:
/home/khanh/Projects/KhoaLuan/my_data/dev/5154139170448018903.wav
🛠 Đang sửa file: data-bin/train.tsv...
✅ Đã sửa xong! Kiểm tra dòng đầu tiên:


IndexError: list index out of range

In [28]:
# thay thế file cũ
!mv data-bin/train_st.tsv data-bin/train.tsv
!mv data-bin/valid_st.tsv data-bin/valid.tsv

In [32]:
!fairseq-train data-bin \
    --config-yaml config.yaml \
    --task speech_to_text \
    --arch s2t_transformer_s \
    --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 10.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 500000 \
    --batch-size 2 \
    --update-freq 8 \
    --max-epoch 50 \
    --save-dir checkpoints \
    --log-format json --log-interval 10 \
    --fp16 \
    --max-source-positions 900000 \
    --max-target-positions 2048 \
    --skip-invalid-size-inputs-valid-test

2026-01-21 21:48:17 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 10, 'log_format': 'json', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': N